In [1]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
import time
#import nzpy

#### Google Reviews Functuon Def

###### Google Reviews v1

In [ ]:
def get_review(cities):
    rev_dict = {'City': [],
                'Review': [],
                'Direc' : []}
    for city in cities:
        print(f'Scraping {city} Google Reviews')
        try:
            driver = webdriver.Chrome('C:/Users/142845/Documents/Python Scripts/Drivers/chromedriver.exe')
            driver.maximize_window()

            driver.get(f"https://www.google.com/search?q=fossil+store+{city}")
            driver.find_element_by_xpath('//*[@id="L2AGLb"]/div').click() #Accept cookies
            time.sleep(2)

            driver.find_element_by_xpath('//*[@id="lu_map"]').click() #Google Maps
            response = BeautifulSoup(driver.page_source, 'html.parser')
            direc = response.find('div', class_='Io6YTe fontBodyMedium').text

            time.sleep(2)

            driver.find_element_by_xpath('//*[@id="pane"]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[2]/div/div[1]/span[1]/span/span[1]/span[2]/span[1]/button').click() #Reviews
            review = BeautifulSoup(driver.page_source, 'html.parser')
            review = review.find('div', class_='fontDisplayLarge').text

            rev_dict['City'].append(city)
            rev_dict['Review'].append(review)
            rev_dict['Direc'].append(direc)

        except:
            rev_dict['City'].append(city)
            rev_dict['Review'].append(999)
            rev_dict['Direc'].append(999)

        driver.close()

    return (pd.DataFrame(rev_dict))

###### Google Reviews words and full reviews Functions

In [2]:
#Get the most mentioned words in the Google's reviews
def get_reviews_words(result_set):
    rev_words = {'Word': []}
    for result in result_set[1:-1]:
        Word = result.find('span', class_='uEubGf gm2-body-2').text
        rev_words['Word'].append(Word)
    return list(rev_words.values())

# Get the total reviews per Store
def get_review_summary(result_set):
    rev_dict = {'Review Rate': [],
        'Review Time': [],
        'Review Text' : []}
    for result in result_set:
        review_rate = result.find('span', class_='ODSEW-ShBeI-H1e3jb')["aria-label"]
        review_time = result.find('span',class_='ODSEW-ShBeI-RgZmSc-date').text
        review_text = result.find('span',class_='ODSEW-ShBeI-text').text
        rev_dict['Review Rate'].append(review_rate)
        rev_dict['Review Time'].append(review_time)
        rev_dict['Review Text'].append(review_text)

    return(pd.DataFrame(rev_dict))

###### Google Reviews v2. Include the most common words

In [ ]:
#Google Reviews with the most common Words
def get_review(cities):
    rev_dict = {'City': [],
                'Review': [],
                'Direc' : [],
                'Rev_Words' : []}
                
    for city in cities:
        print(f'Scraping {city} Google Reviews')
        try:
            driver = webdriver.Chrome('C:/Users/142845/Documents/Python Scripts/Drivers/chromedriver.exe')
            driver.maximize_window()

            driver.get(f"https://www.google.com/search?q=fossil+store+{city}")
            driver.find_element_by_xpath('//*[@id="L2AGLb"]/div').click() #Accept cookies
            time.sleep(2)

            driver.find_element_by_xpath('//*[@id="lu_map"]').click() #Google Maps
            response = BeautifulSoup(driver.page_source, 'html.parser')
            direc = response.find('div', class_='QSFF4-text gm2-body-2').text

            #Stars Reviews
            driver.find_element_by_xpath('//*[@id="pane"]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[2]/div/div[1]/span[1]/span/span[1]/span[2]/span[1]/button').click() #Reviews
            review = BeautifulSoup(driver.page_source, 'html.parser')
            review_stars = review.find('div', class_='gm2-display-2').text

            #Most common review words. Use the function built above
            reviews_words = BeautifulSoup(driver.page_source, 'html.parser')
            reviews_words = reviews_words.find_all('div', class_='e2moi')

            reviews_words = get_reviews_words(reviews_words)            

            #Append
            rev_dict['City'].append(city)
            rev_dict['Review'].append(review_stars)
            rev_dict['Direc'].append(direc)
            rev_dict['Rev_Words'].append(reviews_words)

        except:
            rev_dict['City'].append(city)
            rev_dict['Review'].append(999)
            rev_dict['Direc'].append(999)
            rev_dict['Rev_Words'].append(999)

        driver.close()

    return (pd.DataFrame(rev_dict))

In [ ]:
stores['City_Add'] = stores.CITY_GEO + ' ' + stores.ADDR_1
store_list = stores.City_Add.dropna().drop_duplicates(keep='first')
reviews = get_review(store_list)